Handling missing irrigation data

Proxy approach
- Calculate NDMI changes between consecutive readings
- Set thresholds for significant moisture changes
- Identify patterns that don't align with precipitation events
- These misalignments likely indicate irrigation events

In [12]:
import rasterio
import numpy as np
import pandas as pd
from datetime import datetime
import glob
import os

# Read all NDMI files in chronological order
def read_tif_files(directory_path):
    # Get all TIF files
    tif_files = glob.glob(os.path.join(directory_path, '*NDMI*.tif'))
    
    # Extract dates from filenames 
    dates = [os.path.basename(f).split('_to')[0].strip('NDMI_') for f in tif_files] 
    
    # Sort files by date
    sorted_files = sorted(zip(dates, tif_files))
    
    # Store data in a dictionary
    ndmi_data = {}
    for date, file in sorted_files:
        with rasterio.open(file) as src:
            ndmi_data[date] = src.read(1)  # Read the first band
            
    return ndmi_data, src.meta

In [13]:
def calculate_ndmi_changes(ndmi_data):
    dates = sorted(ndmi_data.keys())
    ndmi_changes = {}
    
    for i in range(1, len(dates)):
        current_date = dates[i]
        previous_date = dates[i-1]
        
        change = ndmi_data[current_date] - ndmi_data[previous_date]
        ndmi_changes[current_date] = change
    
    return ndmi_changes

In [ ]:
def identify_irrigation_events(ndmi_changes, precipitation_data, threshold=0.1):
    irrigation_events = {}
    
    for date, change in ndmi_changes.items():
        # Get precipitation for this date
        precip = precipitation_data.get(date, 0)
        
        # Create a mask for significant positive changes
        significant_increase = change > threshold
        
        if precip < 5:  # Assuming less than 5mm rain is negligible
            irrigation_mask = significant_increase
            
            irrigation_amount = change[irrigation_mask] * 100  # Convert NDMI change to mm of water
            
            irrigation_events[date] = {
                'area_irrigated': np.sum(irrigation_mask),
                'average_amount': np.mean(irrigation_amount) if len(irrigation_amount) > 0 else 0
            }
    
    return irrigation_events